In [3]:
import pydot
from IPython.display import SVG, display, clear_output
import os
from pydrake.all import MinimumDistanceLowerBoundConstraint, MultibodyPlant, Parser, DiagramBuilder, AddMultibodyPlantSceneGraph, MeshcatVisualizer, StartMeshcat, RigidTransform, Role, RollPitchYaw, RotationMatrix, Solve, InverseKinematics, MeshcatVisualizerParams, MinimumDistanceLowerBoundConstraint, DoDifferentialInverseKinematics, DifferentialInverseKinematicsStatus, DifferentialInverseKinematicsParameters, DepthImageToPointCloud
from pydrake.geometry import CollisionFilterDeclaration 
from manipulation.scenarios import AddMultibodyTriad
from manipulation.meshcat_utils import (  # TODO(russt): switch to pydrake version
    _MeshcatPoseSliders,
)
from manipulation.scenarios import AddIiwa, AddShape, AddWsg
import numpy as np
import time

In [ ]:
class Kinematics:
    def __init__(self):
        self.meshcat = StartMeshcat()
        self.urdf_path = "../assets/devkit_base_descr.urdf"
        self.urdf_path = os.path.abspath(self.urdf_path)
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=0.0)
        self.parser = Parser(self.plant)
        self.model_instances = self.parser.AddModelsFromUrl(f"file://{self.urdf_path}")
        self.kinematic_chain_joints = [
            "pillar_platform_joint",
            "joint1",
            "joint2",
            "joint3",
            "joint4",
            "joint5",
            "joint6",
        ]

        # We'll use the first model instance returned
        self.model_instance = self.model_instances[0] if self.model_instances else None

        links_to_ignore = [
            "devkit_base_link",
            "pillar_platform",
            "piper_angled_mount",
            "pan_tilt_base",
            "pan_tilt_head",
            "pan_tilt_pan",
            "base_link",
            "link1",
            "link2",
            "link3",
            "link4",
            "link5",
            "link6"
        ]
        bodies = []
        for link_name in links_to_ignore:
            bodies.extend(self.plant.GetBodiesWeldedTo(self.plant.GetBodyByName(link_name)))
        
        arm_geoms = self.plant.CollectRegisteredGeometries(bodies)
        decl = CollisionFilterDeclaration().ExcludeWithin(arm_geoms)
        manager = self.scene_graph.collision_filter_manager()
        manager.Apply(decl)

        # Finalize the plant
        self.plant.Finalize()

        self.joint_indices = []
        for joint_name in self.kinematic_chain_joints:
            try:
                joint = self.plant.GetJointByName(joint_name)
                if joint.num_positions() > 0:  # Only add if it's an actuated joint
                    self.joint_indices.extend(
                        range(joint.position_start(), joint.position_start() + joint.num_positions())
                    )
            except RuntimeError:
                print(f"Warning: Joint {joint_name} not found in the URDF")


        self.meshcat.Delete()
        self.meshcat.DeleteAddedControls()
        self.visualizer = MeshcatVisualizer.AddToBuilder(self.builder, self.scene_graph, self.meshcat)
        self.end_effector_link = self.plant.GetBodyByName("link6")
        self.end_effector_frame = self.plant.GetFrameByName("link6")

        self.diagram = self.builder.Build()
        self.diagram_context = self.diagram.CreateDefaultContext()
        self.plant_context = self.plant.GetMyContextFromRoot(self.diagram_context)

        # for body_name in [
        #     "devkit_base_link",
        #     "pillar_platform_link",
        #     "pan_tilt_head_link",
        #     "pan_tilt_pan_link",
        #     "link1",
        #     "link2",
        #     "link3",
        #     "link4",
        #     "link5",
        #     "link6",
        #     "link7",
        #     "link8",
        # ]:
        #     AddMultibodyTriad(self.plant.GetBodyByName(body_name), self.scene_graph)



    def set_joint_positions(self, positions):
        full_q = self.plant.GetPositions(self.plant_context)
        for i, idx in enumerate(self.joint_indices):
            full_q[idx] = positions[i]
        self.plant.SetPositions(self.plant_context, full_q)
    
    def get_joint_positions(self):
        full_q = self.plant.GetPositions(self.plant_context)
        return full_q[self.joint_indices]
    
    def get_end_effector_position(self):
        return self.plant.EvalBodyPoseInWorld(self.plant_context, self.end_effector_link)
    
    def forward_kinematics(self, positions):
        self.set_joint_positions(positions)
        return self.get_end_effector_position()

    def compute_ik_errors(self, plant, plant_context, body_frame, 
                      target_position, target_rotation):
            """
            Args:
            plant:            your MultibodyPlant
            plant_context:    the Context you used to solve IK in
            body_frame:       the Frame (or Body) whose pose you constrained
            target_position:  length-3 iterable of floats [x, y, z]
            target_rotation:  RotationMatrix
            Returns:
            pos_err:   np.ndarray shape (3,) giving achieved_xyz – target_xyz  
            rpy_err:   np.ndarray shape (3,) giving (roll, pitch, yaw) error
                        (i.e. the Euler‐angle difference you still have to correct)
            """
            # 1) build the “goal” RigidTransform
            X_WG = RigidTransform(
                target_rotation,
                np.asarray(target_position),
            )

            # 2) ask the plant what the *achieved* world→body pose is
            #    (if you passed in a Body, use .body_frame(); if you passed in a Frame, use it directly)
            X_WB = plant.EvalBodyPoseInWorld(
                plant_context,
                body_frame.body() if hasattr(body_frame, "body") else body_frame
            )

            # 3) translation error:
            p_WG = X_WG.translation()
            p_WB = X_WB.translation()
            pos_err = p_WB - p_WG

            # 4) rotation error as RPY difference:
            R_WG = X_WG.rotation()
            R_WB = X_WB.rotation()
            #  R_error = R_goalᵀ * R_actual
            R_err = R_WG.inverse().multiply(R_WB)
            # wrap that into RollPitchYaw to get (roll,pitch,yaw) offsets
            rpy_err = RollPitchYaw(R_err)
            rot_err = np.array([
                rpy_err.roll_angle(),
                rpy_err.pitch_angle(),
                rpy_err.yaw_angle(),
            ])

            return pos_err, rot_err
    
    def interactive_inverse_kinematics(self):
        q0 = self.plant.GetPositions(self.plant_context)
        gripper_frame = self.plant.GetFrameByName("link6")
        def my_callback(context, pose):
            ik = InverseKinematics(self.plant, self.plant_context)
            ik.AddPositionConstraint(
                gripper_frame,
                [0,0,0],
                self.plant.world_frame(),
                pose.translation(),
                pose.translation(),
            )
            ik.AddOrientationConstraint(
                gripper_frame,
                RotationMatrix(),
                self.plant.world_frame(),
                pose.rotation(),
                0.0,
            )
            ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.1)
            prog = ik.get_mutable_prog()
            q = ik.q()
            prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
            prog.SetInitialGuess(q, q0)
            result = Solve(ik.prog())
            q_sol_joints = result.GetSolution(q)
            q_joints = np.array([])
            for i, idx in enumerate(self.joint_indices):
                q_joints = np.append(q_joints, q_sol_joints[idx])
            print(q_joints)
            self.plant.SetPositions(self.plant_context, q_sol_joints)

            # query_port = self.scene_graph.GetOutputPort("query")
            # query_object = query_port.Eval(self.scene_graph.GetMyContextFromRoot(self.diagram_context))
            # all_sd_pairs: list[SignedDistancePair] = query_object.ComputeSignedDistancePairwiseClosestPoints(0.0)
            # #    Now filter to only the negative ones:
            # penetrating = [sd for sd in all_sd_pairs if sd.distance < 0.0]
            # penetration_pairs: list[PenetrationAsPointPair] = query_object.ComputePointPairPenetration()
            # print(len(penetration_pairs))
            # for pp in penetration_pairs:
            #     print("PENETRATING")
            #     print(f"Penetrating pair: {pp.id_A} vs {pp.id_B}")
            #         print("  witness A (W):  ", pp.p_WCa)           # contact point on A, in World
            #         print("  witness B (W):  ", pp.p_WCb)           # contact point on B, in World
            #     print("  normal A→B (W):", pp.nhat_BA_W)        # surface normal from A to B
            #     break
            prog = ik.get_mutable_prog()
            tol = 1e-6
            constraints  = prog.GetAllConstraints()
            for constraint in constraints:
                print(constraint)
                print(constraint.evaluator().CheckSatisfied(q_sol_joints, tol))

            pos_err, rot_err = self.compute_ik_errors(self.plant, self.plant_context, self.end_effector_frame, pose.translation(), pose.rotation())
            print(pos_err, rot_err)
            if result.is_success():
                print("IK Success")
                return q_joints
            else:
                print("IK Failure")
                # return None
            clear_output(wait=True)
        self.meshcat.DeleteAddedControls()
        sliders = _MeshcatPoseSliders(self.meshcat)
        sliders.SetPose(
            self.plant.EvalBodyPoseInWorld(self.plant_context, self.plant.GetBodyByName("link6"))
        )
        sliders.Run(self.visualizer, self.diagram_context, my_callback)

    def differential_ik(self):
        diff_ik_params = DifferentialInverseKinematicsParameters(
            self.plant.num_positions(),
            self.plant.num_velocities(),
        )
        init_pos = self.plant.EvalBodyPoseInWorld(self.plant_context, self.end_effector_link)
        dt = 0.05
        diff_ik_params.set_time_step(dt)

        q_lower = self.plant.GetPositionLowerLimits()
        q_upper = self.plant.GetPositionUpperLimits()
        diff_ik_params.set_joint_position_limits((q_lower, q_upper)) 

        v_lower = self.plant.GetVelocityLowerLimits()
        v_upper = self.plant.GetVelocityUpperLimits()
        diff_ik_params.set_joint_velocity_limits((v_lower, v_upper))

        flags = np.ones((6,1), dtype=bool)
        flags[0:3] = False
        
        print(flags)
        diff_ik_params.set_end_effector_velocity_flag(flags)

        V_des = np.zeros((6,1))
        V_des[3] = -0.01
        V_des[4] = 0.01
        V_des[5] = -0.05
        print(V_des)
        x = RigidTransform(init_pos.rotation(), np.array([init_pos.translation()[0] - .3, init_pos.translation()[1], init_pos.translation()[2] - .4]))

        # 5) Call differential IK
        for i in range(500):
            result = DoDifferentialInverseKinematics(
                self.plant,
                self.plant_context,
                x,
                self.end_effector_frame,
                diff_ik_params
            )  # wrapper that pulls q̇, v̇ from the context :contentReference[oaicite:2]{index=2}

            # 6) Check and apply
            if result.status == DifferentialInverseKinematicsStatus.kSolutionFound:
                v_next = result.joint_velocities  # shape (n_v,1)
                # integrate: q_new = q_old + v_next * dt
                q_old = self.plant.GetPositions(self.plant_context)
                q_new = q_old + v_next.flatten() * dt
                self.plant.SetPositions(self.plant_context, q_new)
                print("Applied joint velocities:", v_next.T)
            else:
                print("No solution (or stuck); status =", result.status)

            self.diagram.ForcedPublish(self.diagram_context)


            time.sleep(dt)
        
        


kinematics = Kinematics()
print(kinematics.forward_kinematics([0, 0, 0, 0, 0, 0, 0]))
# kinematics.interactive_inverse_kinematics()
kinematics.differential_ik()


INFO:drake:Meshcat listening for connections at http://localhost:7000


RigidTransform(
  R=RotationMatrix([
    [0.6428371036290381, -0.00019773666284053773, -0.7660028845233164],
    [-0.00011962288848600904, -0.999999980402306, 0.00015775218376425256],
    [-0.7660029007048169, -9.777479266873963e-06, -0.6428371146847442],
  ]),
  p=[-0.48309420322314856, 6.493654127547697e-05, 1.7761612095781294],
)
[[False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]]
[[ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [-0.01]
 [ 0.01]
 [-0.05]]
Applied joint velocities: [ 0.00000000e+00  0.00000000e+00 -1.00000000e-01  2.09857928e-04
  4.68762169e-01 -2.22334434e-01  6.42360002e-07 -8.10218739e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00]
Applied joint velocities: [ 0.00000000e+00  0.00000000e+00 -1.00000000e-01  2.06753196e-04
  4.91203326e-01 -2.26511059e-01 -7.20068106e-07 -7.98211045e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00]
Applied joint velocities: [ 0.00000000e+00  0.00000000e+00 -1.00000000e-01  2.02141409e-04
  5.14157025e-01 -2.31130579e-01 -1.82799809e-06 -7.

KeyboardInterrupt: 